<a href="https://colab.research.google.com/github/AntonioDeFA/etl_pyspark/blob/develop/Dados_IBGE_Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install pyspark findspark unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.1 MB/s eta 0:00:00


In [27]:
import unicodedata
import seaborn as sns
import matplotlib.pyplot as plt
from unidecode import unidecode
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.functions import col
from pyspark.sql import SparkSession, Row

In [28]:
remove_accents_udf = udf(lambda name: unidecode(name) if name else name, StringType())

In [16]:
spark = SparkSession.builder \
    .appName("Dados IBGE") \
    .getOrCreate()

input_path_ibge = "/content/cities-brazil-ibge.csv"
data_ibge = spark.read.csv(input_path_ibge,
                           header=True,
                           inferSchema=True,
                           sep=",",
                           encoding="UTF-8")

input_path_covid = "/content/cases-brazil-cities-time_2022.csv"
data_covid = spark.read.csv(input_path_covid,
                            header=True,
                            inferSchema=True,
                            sep=",",
                            encoding="UTF-8")

In [17]:
windowSpec = Window.partitionBy("city").orderBy(col("date").desc())

last_cases_covid_data = data_covid.withColumn("row_num",
                                       row_number().
                                       over(windowSpec)).\
                                       filter(col("row_num") == 1).\
                                       drop("row_num")


In [29]:
remove_accents_data = last_cases_covid_data.withColumn("city", remove_accents_udf("city"))

In [ ]:
remove_accents_data.show()

In [34]:
filter_city_name_data = remove_accents_data.filter(
    upper(col("city")).startswith("CO")
).select("*")

In [35]:
filter_city_name_data.show()

+--------+----------+-----+--------------------+-------+---------+------+--------+----------+---------------------------+-------------------------------+--------------------+
|epi_week|      date|state|                city| ibgeID|newDeaths|deaths|newCases|totalCases|deaths_per_100k_inhabitants|totalCases_per_100k_inhabitants|deaths_by_totalCases|
+--------+----------+-----+--------------------+-------+---------+------+--------+----------+---------------------------+-------------------------------+--------------------+
|     252|2022-12-31|   BA|          Coaraci/BA|2908002|        0|    59|       0|      2798|                    365.823|                       17348.71|               0.021|
|     252|2022-12-31|   AM|            Coari/AM|1301209|        0|   249|       0|     11799|                    287.154|                      13606.956|               0.021|
|     252|2022-12-31|   PI|   Cocal de Telha/PI|2202711|        0|    14|       0|       631|                    285.249|    

In [ ]:
filter_state_name_data = filter_city_name_data.filter(
    (col("state") == "PB")
).select("*")

In [ ]:
filter_state_name_data.show()

In [23]:

filtered_percente_data = filter_city_name_data.filter(
    (((col("deaths")/col("totalCases"))*100)>0)
).select("*")

In [36]:
filtered_data.show()

<bound method DataFrame.show of DataFrame[epi_week: int, date: date, state: string, city: string, ibgeID: int, newDeaths: int, deaths: int, newCases: int, totalCases: int, deaths_per_100k_inhabitants: double, totalCases_per_100k_inhabitants: double, deaths_by_totalCases: double]>